In [13]:
from produto import Produto
from caixa import Caixa
from estoque import Estoque
from onda import Onda

from pyspark.sql import SparkSession
from pyspark import SparkConf

# Configuração inicial do PySpark
conf = SparkConf()
conf.setAppName("MeuAplicativoPySpark")
conf.setMaster("local[*]")  # Usar todos os núcleos locais para execução

# Criação da SparkSession
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()

In [14]:
# Verificar se a sessão Spark está funcionando
print("SparkSession Criada!")
spark.sparkContext.setLogLevel("WARN")  # Define o nível de log para evitar muitas mensagens de depuração

# Leitura de dois arquivos CSV e salvando em variáveis
caixas_df = spark.read.csv("../data/caixas.csv", header=True, inferSchema=True)
estoque_df = spark.read.csv("../data/estoque.csv", header=True, inferSchema=True)

# Mostrando o conteúdo dos DataFrames
caixas_df.show(1)
estoque_df.show(1)


SparkSession Criada!
+-------+--------+-----+-------------+-----+
|ONDA_ID|CAIXA_ID|PECAS|  CLASSE_ONDA|  SKU|
+-------+--------+-----+-------------+-----+
|      4|      12|    1|CLASSE_ONDA_1|SKU_1|
+-------+--------+-----+-------------+-----+
only showing top 1 row

+-----+--------+---------+-----+
|ANDAR|CORREDOR|      SKU|PECAS|
+-----+--------+---------+-----+
|    0|       2|SKU_17028|  193|
+-----+--------+---------+-----+
only showing top 1 row



In [15]:
# Criando uma visão temporária para utilizar SQL puro
caixas_df.createOrReplaceTempView("caixas")
estoque_df.createOrReplaceTempView("estoque")

In [16]:

# Agregando as quantidades de cada SKU por classe de onda e caixa
agregado_caixa = spark.sql("""
    SELECT
        CLASSE_ONDA,
        CAIXA_ID,
        SKU,
        PECAS
    FROM caixas

""")
agregado_caixa.createOrReplaceTempView("agregado_caixa")

# Agregando as quantidades de cada SKU por andar
agregado_estoque = spark.sql("""
    SELECT
        ANDAR,
        SKU,
        SUM(PECAS) AS TOTAL_PECAS_DISPONIVEIS
    FROM estoque
    GROUP BY
        ANDAR, SKU
""")
agregado_estoque.createOrReplaceTempView("agregado_estoque")

# Verificando quantas peças são necessárias em cada caixa e quantas temos no andar
dados_necessarios_disponiveis = spark.sql("""
    SELECT
        ac.CLASSE_ONDA,
        ac.CAIXA_ID,
        ae.ANDAR,
        ac.SKU,
        ac.PECAS AS TOTAL_PECAS_NECESSARIAS,
        ae.TOTAL_PECAS_DISPONIVEIS,
        (ae.TOTAL_PECAS_DISPONIVEIS - ac.PECAS) AS DIFF
        
    FROM agregado_caixa ac
    LEFT JOIN agregado_estoque ae
    ON ac.SKU = ae.SKU
    ORDER BY
        ac.CLASSE_ONDA, ac.CAIXA_ID, ae.ANDAR
""")
dados_necessarios_disponiveis.show()

+-------------+--------+-----+------+-----------------------+-----------------------+----+
|  CLASSE_ONDA|CAIXA_ID|ANDAR|   SKU|TOTAL_PECAS_NECESSARIAS|TOTAL_PECAS_DISPONIVEIS|DIFF|
+-------------+--------+-----+------+-----------------------+-----------------------+----+
|CLASSE_ONDA_1|      12|    0| SKU_2|                      1|                     84|  83|
|CLASSE_ONDA_1|      12|    0| SKU_3|                      1|                    852| 851|
|CLASSE_ONDA_1|      12|    0| SKU_4|                      1|                    670| 669|
|CLASSE_ONDA_1|      12|    0| SKU_6|                      4|                    233| 229|
|CLASSE_ONDA_1|      12|    0| SKU_9|                      4|                    152| 148|
|CLASSE_ONDA_1|      12|    0|SKU_11|                      1|                      1|   0|
|CLASSE_ONDA_1|      12|    0|SKU_12|                      2|                    132| 130|
|CLASSE_ONDA_1|      12|    0|SKU_14|                      8|                    271| 263|

In [17]:
# Filtrando os casos onde o DIFF > 0
caixas_easy = dados_necessarios_disponiveis.filter("DIFF >= 0")
caixas_easy.count()

23299

In [18]:
caixas_hard = dados_necessarios_disponiveis.filter("DIFF < 0")
caixas_hard.count()

218

In [28]:
caixas = {}

for row in caixas_df.collect():
    onda_id = row['ONDA_ID']
    caixa_id = row['CAIXA_ID']
    classe_onda = row['CLASSE_ONDA']
    classe_onda = int(classe_onda.split('_')[-1])
    sku = row['SKU']
    sku = int(sku.split('_')[1])
    qtd = row['PECAS']
    
    # Criar ou buscar a caixa existente
    if caixa_id not in caixas:
        caixa = Caixa(caixa_id, classe_onda, onda_id)
        caixas[caixa_id] = caixa
    else:
        caixa = caixas[caixa_id]
    
    # Adicionar o produto à caixa
    produto = Produto(sku, qtd)
    caixa.add_produto(produto)

print(len(caixas))
print(caixas[12].get_total_itens())

2072
68
6


In [20]:
estoques = {}
for row in estoque_df.collect():

    andar = row['ANDAR']
    corredor = row['CORREDOR']
    sku = row['SKU']
    sku = int(sku.split('_')[1])
    qtd = row['PECAS']
    
    # Criar ou buscar o estoque existente
    if (andar, corredor) not in estoques:
        estoque = Estoque(andar, corredor)
        estoques[(andar, corredor)] = estoque
    
    else:
        estoque = estoques[(andar, corredor)]

    # Adicionar o produto ao estoque
    produto = Produto(sku, qtd)
    estoque.add_produto(produto)

print(len(estoques))
print(estoques[(0, 2)].get_total_itens())

502
1615


In [30]:
for caixa in caixas.values():

    total_prod_n = 0
    total_prod_s = 0

    for produto in caixa.produtos:
        aux = 0

        sku = produto.sku
        qtd = produto.qtd

        for estoque in estoques.values():

            produto_estoque = estoque.get_produto(sku)
            

            if produto_estoque is not None and produto_estoque.qtd >= qtd: 
                total_prod_s += 1
                aux = 1
                produto_estoque.qtd -= qtd
                break
        
        if aux == 0:
            total_prod_n += 1
    
    print(f"Caixa {caixa.id_caixa} - Produtos sem estoque: {total_prod_n} - Produtos com estoque: {total_prod_s}")


Caixa 12 - Produtos sem estoque: 0 - Produtos com estoque: 17
Caixa 13 - Produtos sem estoque: 0 - Produtos com estoque: 31
Caixa 45 - Produtos sem estoque: 0 - Produtos com estoque: 27
Caixa 46 - Produtos sem estoque: 0 - Produtos com estoque: 26
Caixa 48 - Produtos sem estoque: 0 - Produtos com estoque: 25
Caixa 49 - Produtos sem estoque: 0 - Produtos com estoque: 19
Caixa 184 - Produtos sem estoque: 1 - Produtos com estoque: 9
Caixa 249 - Produtos sem estoque: 21 - Produtos com estoque: 1
Caixa 490 - Produtos sem estoque: 1 - Produtos com estoque: 30
Caixa 516 - Produtos sem estoque: 1 - Produtos com estoque: 4
Caixa 525 - Produtos sem estoque: 0 - Produtos com estoque: 13
Caixa 538 - Produtos sem estoque: 1 - Produtos com estoque: 5
Caixa 575 - Produtos sem estoque: 0 - Produtos com estoque: 8
Caixa 629 - Produtos sem estoque: 0 - Produtos com estoque: 11
Caixa 630 - Produtos sem estoque: 0 - Produtos com estoque: 26
Caixa 631 - Produtos sem estoque: 0 - Produtos com estoque: 15
Ca

In [43]:
caixas_classe1 = {}
caixas_classe2 = {}
caixas_classe3 = {}
caixas_classe4 = {}
caixas_classe5 = {}
caixas_classe6 = {}

for caixa in caixas.values():
    if caixa.classe_onda == 1:
        caixas_classe1[caixa.id_caixa] = caixa
    elif caixa.classe_onda == 2:
        caixas_classe2[caixa.id_caixa] = caixa
    elif caixa.classe_onda == 3:
        caixas_classe3[caixa.id_caixa] = caixa
    elif caixa.classe_onda == 4:
        caixas_classe4[caixa.id_caixa] = caixa
    elif caixa.classe_onda == 5:
        caixas_classe5[caixa.id_caixa] = caixa
    else:
        caixas_classe6[caixa.id_caixa] = caixa

print(f"Caixas classe 1: {len(caixas_classe1)}")
print(f"Caixas classe 2: {len(caixas_classe2)}")
print(f"Caixas classe 3: {len(caixas_classe3)}")
print(f"Caixas classe 4: {len(caixas_classe4)}")
print(f"Caixas classe 5: {len(caixas_classe5)}")
print(f"Caixas classe 6: {len(caixas_classe6)}")

Caixas classe 1: 1133
Caixas classe 2: 416
Caixas classe 3: 321
Caixas classe 4: 153
Caixas classe 5: 47
Caixas classe 6: 2


In [52]:
caixas_andar_0 = {}
caixas_andar_1 = {}
caixas_andar_2 = {}
caixas_varios = {}

for caixa in caixas_classe5.values():

    min_andares = []

    for produto in caixa.produtos:

        #print(f"Produto {produto.sku} - Qtd {produto.qtd}")
        andares_achados = set()
        achou = False

        sku = produto.sku
        qtd = produto.qtd

        for estoque in estoques.values():
            produto_estoque = estoque.get_produto(sku)

            if produto_estoque is not None and produto_estoque.qtd >= qtd-1:
                andares_achados.add(estoque.andar)
                achou = True

        #print(f"Produto {sku} - Andares: {andares_achados}")

        if achou:
            min_andares.extend(andares_achados - set(min_andares))
        
    min_andares.sort()

    if len(min_andares) == 1:
        if min_andares[0] == 0:
            caixas_andar_0[caixa.id_caixa] = min_andares
        elif min_andares[0] == 1:
            caixas_andar_1[caixa.id_caixa] = min_andares
        else:
            caixas_andar_2[caixa.id_caixa] = min_andares    
    else:
        caixas_varios[caixa.id_caixa] = min_andares

print(f"Caixas andar 0: {len(caixas_andar_0)}")
print(f"Caixas andar 1: {len(caixas_andar_1)}")
print(f"Caixas andar 2: {len(caixas_andar_2)}")
print(f"Caixas varios: {len(caixas_varios)}")


Caixas andar 0: 33
Caixas andar 1: 0
Caixas andar 2: 0
Caixas varios: 14
